`02-rag.ipynb`
```sh
pip install langchain langchain-chroma langchain-openai
```

In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
model = ChatOpenAI(model="gpt-3.5-turbo-1106")

In [2]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [5]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
)

In [12]:
vectorstore.similarity_search_with_score('not annoying')

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.5027285218238831),
 (Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  0.5148149132728577),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  0.5198458433151245),
 (Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
  0.5290757417678833)]

In [21]:
emb = OpenAIEmbeddings().embed_query('not annoying')

In [23]:
vectorstore.similarity_search_by_vector(emb)

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]

In [24]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

In [25]:
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(['cat', 'shark'])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

llm = ChatOpenAI(model='gpt-3.5-turbo-1106')

message = '''
Answer this question using the provided context only.

{question}

Context:
{context}
'''

prompt = ChatPromptTemplate.from_messages(['human', message])

rag_chain = {'context': retriever, 'question': RunnablePassthrough()} | prompt | llm

In [34]:
res = rag_chain.invoke('tell me about cat and parrot')

In [35]:
res.content

"I'm sorry, I cannot provide an accurate answer to your question as the context only mentions parrots and does not provide any information about cats. If you have specific questions about parrots, I would be happy to help."